In [ ]:
import pandas as pd

# Data snapshot

In [ ]:
df = pd.read_csv('crunchbase-investments.csv', encoding='ISO-8859-1', low_memory=False)
df.head(5)

In [ ]:
df.isnull().sum()

In [ ]:
df.describe()

In [ ]:
df.shape

# Unnecessary columns

We can drop/exclude the columns **`funded_month`** and **`funded_year`** because they contain redundant info that is already contained in **`funded_at`**.

# Missing value counts

In [ ]:
use_cols = ['investor_region', 'investor_permalink', 'investor_name',
        'investor_country_code', 'investor_city', 'investor_category_code', 
        'funding_round_type', 'company_category_code', 'funded_at', 
        'company_state_code', 'company_region', 'company_permalink', 
        'company_name', 'company_country_code', 'company_city', 
        'investor_state_code', 'funded_quarter', 'raised_amount_usd']

chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=use_cols)

for chunk in chunk_iter:    
    missing_value_counts = chunk.isnull().sum()
    print(missing_value_counts)
    
    print('='*50)

# Memory footprint

In [ ]:
use_cols = ['investor_region', 'investor_permalink', 'investor_name',
        'investor_country_code', 'investor_city', 'investor_category_code', 
        'funding_round_type', 'company_category_code', 'funded_at', 
        'company_state_code', 'company_region', 'company_permalink', 
        'company_name', 'company_country_code', 'company_city', 
        'investor_state_code', 'funded_quarter', 'raised_amount_usd']

total_memory = 0
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=use_cols)
for chunk in chunk_iter:    
    chunk_memory = chunk.memory_usage(deep=True, index=False).sum()
    total_memory += chunk_memory    
    print('Chunk memory usage (MB) =', chunk_memory/(1024*1024))

print('-'*50)
print('Total memory usage across all chunks (MB) =', total_memory/(1024*1024))

# Column types (by reading entire file at once)

In [ ]:
use_cols = ['investor_region', 'investor_permalink', 'investor_name',
        'investor_country_code', 'investor_city', 'investor_category_code', 
        'funding_round_type', 'company_category_code', 'funded_at', 
        'company_state_code', 'company_region', 'company_permalink', 
        'company_name', 'company_country_code', 'company_city', 
        'investor_state_code', 'funded_quarter', 'raised_amount_usd']

df = pd.read_csv('crunchbase-investments.csv', encoding='ISO-8859-1', low_memory=False, usecols=use_cols)
df.dtypes

# Column types (by reading file in chunks)

In [ ]:
# A column's datatype may not be consistent across all chunks.
# To get a column's datatype, we gather its datatype in each chunk, 
# then call max() to get its type with the highest precedence.
# Example: 'investor_category_code' is listed as an object type 
# in one chunk and as a float in another chunk because all the
# values in that chunk are NaN. Between object and float, the 
# object type has a higher precedence, so that's the datatype
# returned.

use_cols = ['investor_region', 'investor_permalink', 'investor_name',
        'investor_country_code', 'investor_city', 'investor_category_code', 
        'funding_round_type', 'company_category_code', 'funded_at', 
        'company_state_code', 'company_region', 'company_permalink', 
        'company_name', 'company_country_code', 'company_city', 
        'investor_state_code', 'funded_quarter', 'raised_amount_usd']

chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=use_cols)
datatypes = pd.Series()
for chunk in chunk_iter:
    d_types = chunk.dtypes
    datatypes = datatypes.append(d_types)
    
colnames = datatypes.index
datatypes = datatypes.groupby(by=colnames).max().sort_values()
numeric_cols = datatypes[datatypes != 'object']
string_cols = datatypes[datatypes == 'object']

print('Numeric columns: ( n =', numeric_cols.size, ')\n')
print(numeric_cols)
print('\nString columns: ( n =', string_cols.size, ')\n')
print(string_cols)

# Optimize numeric columns

In [ ]:
use_cols = ['investor_region', 'investor_permalink', 'investor_name',
        'investor_country_code', 'investor_city', 'investor_category_code', 
        'funding_round_type', 'company_category_code', 'funded_at', 
        'company_state_code', 'company_region', 'company_permalink', 
        'company_name', 'company_country_code', 'company_city', 
        'investor_state_code', 'funded_quarter', 'raised_amount_usd']

chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=use_cols)
for chunk in chunk_iter:
    print(chunk['raised_amount_usd'].head(2)) 
    print('-'*50)

From the results above, we can see that we should cast **`raised_amount_usd`** to an `int64`. The float type is overkill for this column.

Let's make that change.

In [ ]:
use_cols = ['investor_region', 'investor_permalink', 'investor_name',
        'investor_country_code', 'investor_city', 'investor_category_code', 
        'funding_round_type', 'company_category_code', 'funded_at', 
        'company_state_code', 'company_region', 'company_permalink', 
        'company_name', 'company_country_code', 'company_city', 
        'investor_state_code', 'funded_quarter', 'raised_amount_usd']

chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=use_cols)
for chunk in chunk_iter:
    print('dtype before optimization:', chunk['raised_amount_usd'].dtype)
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0).astype('int')
    print('dtype after optimization:', chunk['raised_amount_usd'].dtype)
    print('-'*50)

# Optimize string columns

By looking at the string column **`funded_quarter`**, we can see that we could optimize it by extracting the number representing the quarter and then cast the column to an `int8` type.

- Example: Extract the `4` from `2012-Q4`	 

In [ ]:
use_cols = ['investor_region', 'investor_permalink', 'investor_name',
        'investor_country_code', 'investor_city', 'investor_category_code', 
        'funding_round_type', 'company_category_code', 'funded_at', 
        'company_state_code', 'company_region', 'company_permalink', 
        'company_name', 'company_country_code', 'company_city', 
        'investor_state_code', 'funded_quarter', 'raised_amount_usd']

total_memory = 0
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=use_cols)
for chunk in chunk_iter:
    # Optimize numeric column
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0).astype('int')

    # Optimize 'funded_quarter' column
    chunk['funded_quarter'] = chunk['funded_quarter'].str.extract(r'Q(\d)', expand=False)
    chunk['funded_quarter'] = chunk['funded_quarter'].fillna(-1)
    chunk['funded_quarter'] = chunk['funded_quarter'].astype('int8')

    chunk_memory = chunk.memory_usage(deep=True).sum()
    total_memory += chunk_memory
    print('Chunk memory usage (MB) =', chunk_memory/(1024*1024))

print('-'*50)
print('Total memory usage (MB) =', total_memory/(1024*1024))

From the results above, we can see that we were able to lower the total memory usage from 53.35 MB to 49.98 MB by optimizing some of the columns.

# Load data into SQLite

In [ ]:
import sqlite3

In [ ]:
# Create and connect to a new database
conn = sqlite3.connect('crunchbase.db')
cur = conn.cursor()
cur.execute('''DROP TABLE IF EXISTS investments''')

use_cols = ['investor_region', 'investor_permalink', 'investor_name',
        'investor_country_code', 'investor_city', 'investor_category_code', 
        'funding_round_type', 'company_category_code', 'funded_at', 
        'company_state_code', 'company_region', 'company_permalink', 
        'company_name', 'company_country_code', 'company_city', 
        'investor_state_code', 'funded_quarter', 'raised_amount_usd']

total_memory = 0
chunk_iter = pd.read_csv('crunchbase-investments.csv', chunksize=5000, encoding='ISO-8859-1', usecols=use_cols)
for chunk in chunk_iter:
    # Optimize numeric column
    chunk['raised_amount_usd'] = chunk['raised_amount_usd'].fillna(0).astype('int')
    
    # Optimize 'funded_quarter' column
    chunk['funded_quarter'] = chunk['funded_quarter'].str.extract(r'Q(\d)', expand=False)
    chunk['funded_quarter'] = chunk['funded_quarter'].fillna(-1)
    chunk['funded_quarter'] = chunk['funded_quarter'].astype('int8')

    # Write chunk out to database table
    chunk.to_sql('investments', conn, if_exists='append', index=False)

# Data load verification

Let's now confirm that the data is in the database table.

In [ ]:
results_df = pd.read_sql('PRAGMA table_info(investments)', conn)
results_df

In [ ]:
results_df = pd.read_sql('SELECT COUNT(*) FROM investments', conn)
results_df

In [ ]:
results_df = pd.read_sql('SELECT * FROM investments LIMIT 3', conn)
results_df

# Data analysis

In [ ]:
query = '''
    SELECT 
        company_name, 
        company_category_code,
        investor_name,
        raised_amount_usd
    FROM 
        investments
'''
results_df = pd.read_sql(query, conn)

In [ ]:
grouped = results_df.groupby(by='company_name').raised_amount_usd.sum()
ncompanies = results_df.nunique().company_name 
ordered = grouped.sort_values()

##  What proportion of the total amount of funds did the top 10% raise?

In [ ]:
top10 = ordered.tail(int(round(ncompanies*.10)))
print(top10.sum(), '/', grouped.sum())

## What proportion of the total amount of funds did the top 1% raise?

In [ ]:
top1 = ordered.tail(int(round(ncompanies*.01)))
print(top1.sum(), '/', grouped.sum())

## What proportion of the total amount of funds did the bottom 10% raise?

In [ ]:
bottom10 = ordered.head(int(round(ncompanies*.10)))
print(bottom10.sum(), '/', grouped.sum())

## What proportion of the total amount of funds did the bottom 1% raise?

In [ ]:
bottom1 = ordered.head(int(round(ncompanies*.01)))
print(bottom1.sum(), '/', grouped.sum())

## Which category of company attracted the most investments?

In [ ]:
results_df.groupby(by='company_category_code').raised_amount_usd.sum().sort_values().tail(1)

## Which investor contributed the most money (across all startups)?

In [ ]:
results_df.groupby(by='investor_name').raised_amount_usd.sum().sort_values().tail(1)

## Which investors contributed the most money per startup?

In [ ]:
#g = results_df.groupby(by=['company_name', 'investor_name'])
#type(g)

## Which funding round was the most popular? Which was the least popular?